In [ ]:
initialDeclaration = []

In [ ]:
def solve(layer, previousOutput, index, Adder = False, prev_non_linear = None):
  changed = False
  newOutput = output_var[layer] + "_" + str(index)

  if layer not in code: # Layer name's verilog not available, this condition shouldnt execute.
    print(layer, " not found")
    return []

  toAdd = [] # This is used for initial declaration.
  path = code[layer]
  changedCode = []
  with open(path, 'r') as file:
    for line in file:
      curLine = line
      if "endmodule" not in curLine and "module" in curLine:
        tempLine = ""
        for ch in curLine:
          if (ch == '('):
            break
          tempLine = tempLine + ch

        tempLine = tempLine + "_" + str(index)
        toAdd.append(tempLine[7:])
        curLine = tempLine + "(\n"

      if (output_var[layer] in curLine):
          curLine = curLine.replace(output_var[layer],newOutput)

      if (Adder == False):
        if(input_var[layer] in curLine and previousOutput != "None"):
            curLine = curLine.replace(input_var[layer],previousOutput)
      else:
        if ("vector_a" in curLine):
          curLine = curLine.replace("vector_a", previousOutput)
        if ("vector_b" in curLine):
          curLine = curLine.replace("vector_b", prev_non_linear)

      changedCode.append(curLine)

  if (previousOutput != "None"):
    toAdd.append(previousOutput)
  else:
    toAdd.append(input_var[layer])

  if (Adder):
    toAdd.append(prev_non_linear)

  toAdd.append(newOutput)

  initialDeclaration.append(toAdd)
  return changedCode, newOutput

In [ ]:
code = {}
code["Conv"] = "/content/3x3 Conv.txt"
code["Relu"] = "/content/ReLu.txt"
code["fc"] = "/content/FC.txt"
code["pool"] = "/content/maxpooling.txt"
code["SoftMax"] = "/content/softmax.txt"
code["Adder"] = "/content/Adder.txt"

In [ ]:
input_var = {}
input_var["Relu"] = "din_relu"
input_var["fc"] = "input_data"
input_var["Conv"] = "input_data"
input_var["Resnet"] = "data_in"
input_var["sigmoid"] = "x"
input_var["tanH"] = "phase"
input_var["Midas"] = "data_in"
input_var["DenseNet"] = "input_data"
input_var["CNN"] = "rst"
input_var["pool"] = "in_data"
input_var["SoftMax"] = "Datain"
input_var["Adder"] = ""
output_var = {}
output_var["SoftMax"] = "Dataout"
output_var["Relu"] = "dout_relu"
output_var["fc"] = "output_data"
output_var["Conv"] = "output_data"
output_var["Resnet"] = "data_out"
output_var["sigmoid"] = "out"
output_var["tanH"] = "tanh"
output_var["Midas"] = "data_out"
output_var["DenseNet"] = "output_data"
output_var["CNN"] = "numberOfTimes_PatterDetected"
output_var["pool"] = "out_data"
output_var["Adder"] = "result_vector"

In [ ]:
params = {"vgg16" : {"parameter WIDTH" : [224, 224, 112, 112, 56, 56, 56, 28, 28, 28, 14, 14, 14],
                     "parameter HEIGHT" : [224, 224, 112, 112, 56, 56, 56, 28, 28, 28, 14, 14, 14],
                     "parameter CHANNELS" : [3, 64, 64, 128, 128, 256, 256, 256, 512, 512, 512, 512, 512],
                     "parameter FILTERS" : [64, 64, 128, 128, 256, 256, 256, 512, 512, 512, 512, 512, 512],
                     "parameter KERNEL_SIZE" : [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
                     "parameter STRIDE": [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                     "parameter PADDING" : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]},
          "alexnet" : {"parameter WIDTH" : [227, 27, 13, 13, 13],
                       "parameter HEIGHT" : [227, 27, 13, 13, 13],
                       "parameter CHANNELS" : [3, 96, 256, 384, 384],
                       "parameter FILTERS" : [96, 256, 384, 384, 256],
                       "parameter KERNEL_SIZE" : [11, 5, 3, 3, 3],
                       "parameter STRIDE" : [4, 1, 1, 1, 1],
                       "parameter PADDING" : [0, 2, 1, 1, 1]},
          "resnet" : {"parameter WIDTH" : [227, 56, 56, 56, 56, 56, 28, 28, 28, 28, 14, 14, 14, 14, 7, 7, 7],
                      "parameter HEIGHT": [227, 56, 56, 56, 56, 56, 28, 28, 28, 28, 14, 14, 14, 14, 7, 7, 7],
                      "parameter CHANNELS" : [3, 64, 64, 64, 64, 64, 128, 128, 128, 128, 256, 256, 256, 256, 512, 512, 512],
                      "parameter FILTERS" : [64, 64, 64, 64, 64, 128, 128, 128, 128, 256, 256, 256, 256, 512, 512, 512, 512],
                      "parameter KERNEL_SIZE": [7, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
                      "parameter STRIDE" : [2, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1],
                      "parameter PADDING" : [3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
          }

In [ ]:
params_index = {"vgg16" : {"parameter WIDTH" : 0,
                           "parameter HEIGHT" : 0,
                           "parameter CHANNELS" : 0,
                           "parameter FILTERS" : 0,
                           "parameter KERNEL_SIZE" : 0,
                           "parameter STRIDE" : 0,
                           "parameter PADDING" : 0},
                "alexnet" : {"parameter WIDTH" : 0,
                           "parameter HEIGHT" : 0,
                           "parameter CHANNELS" : 0,
                           "parameter FILTERS" : 0,
                           "parameter KERNEL_SIZE" : 0,
                           "parameter STRIDE" : 0,
                           "parameter PADDING" : 0},
                "resnet" : {"parameter WIDTH" : 0,
                           "parameter HEIGHT" : 0,
                           "parameter CHANNELS" : 0,
                           "parameter FILTERS" : 0,
                           "parameter KERNEL_SIZE" : 0,
                           "parameter STRIDE" : 0,
                           "parameter PADDING" : 0}
                }
temp = 0

In [ ]:
def change(lines, model): # Parameters change
   newCode = []
   for j in range(len(lines)):
    paramFound = False
    for key in params[model]:
       value = params[model][key]
       if(key in lines[j]):
          paramFound = True
          s = ''
          for k in range(len(lines[j])):
            s+=lines[j][k]
            if(lines[j][k] == "="):
              s += " "
              index = params_index[model][key]
              s+=str(value[index])
              s += ";\n"
              params_index[model][key] = index + 1
              break
          newCode.append(s)
    if (paramFound == False):
      newCode.append(lines[j])
   return newCode

In [ ]:
finalCode = []

In [ ]:
skip = True
previousOutput = "None"
idx_module = 1
model = "vgg16"
modelPath = model + ".txt"
prev_non_linear = None
with open(modelPath, 'r') as file:

    for line in file:
      line = line.strip()
      if (len(line) == 0 or skip):
        skip = False
        continue

      save = False
      layer = ""
      secondString = ""
      cntComma = 0
      for chr in line:
        if (chr == ','):
          if (cntComma == 1):
            break
          else:
            cntComma = 1
            continue
        if (cntComma == 0):
          layer = layer + chr
        else:
          secondString = secondString + chr

      if (secondString == " Save"):
        save = True

      if (layer == "Dropout"):
        continue

      if (layer == "Adder"):
        lines, previousOutput = solve(layer, previousOutput, idx_module, True, prev_non_linear)
      else:
        lines, previousOutput = solve(layer, previousOutput, idx_module)

      if (save):
        prev_non_linear = previousOutput

      idx_module = idx_module + 1

      if (layer == "Conv"):
        lines = change(lines, model)

      finalCode.append(lines)

In [ ]:
first = "module top(input inp, input clk, input rst, output out);\n"
space = "    "
second = space + "wire output_data;\n"
third = space + "wire dout_relu;\n"
fourth = space + "wire Start;\n"
fifth = space + "wire [2:0] N;\n"

In [ ]:
with open('output_file.txt', 'w') as output_file:
  output_file.write(first)
  output_file.write(second)
  output_file.write(third)
  output_file.write(fourth)
  output_file.write(fifth)

  for i in range(len(initialDeclaration)):
    string = initialDeclaration[i][0] + " uut"
    if (i):
      string = string + str(i)

    layer = initialDeclaration[i][0][:4]

    if (layer == "vect"):
      string = space + string + "(.input_data1(" + initialDeclaration[i][1] +").input_data2(" + initialDeclaration[i][2] + "), .output_data(" + initialDeclaration[i][3] + "));\n"
    else:
      if (layer == "Conv"):
        string = string + "(.clk(clk),.rst(rst),.input_data("
      else:
        string = string + "(.input_data("

      string = space + string + initialDeclaration[i][1] + "), .output_data(" + initialDeclaration[i][2] + "));\n"

    output_file.write(string)

  output_file.write("endmodule\n")

  for i in finalCode:
    for j in i:
      output_file.write(j)

In [ ]:
for i in initialDeclaration:
  print(i)

['Conv2D_1', 'input_data', 'output_data_1']
['relu_2', 'output_data_1', 'dout_relu_2']
['Conv2D_3', 'dout_relu_2', 'output_data_3']
['relu_4', 'output_data_3', 'dout_relu_4']
['max_pooling_3d_5', 'dout_relu_4', 'out_data_5']
['Conv2D_6', 'out_data_5', 'output_data_6']
['relu_7', 'output_data_6', 'dout_relu_7']
['Conv2D_8', 'dout_relu_7', 'output_data_8']
['relu_9', 'output_data_8', 'dout_relu_9']
['max_pooling_3d_10', 'dout_relu_9', 'out_data_10']
['Conv2D_11', 'out_data_10', 'output_data_11']
['relu_12', 'output_data_11', 'dout_relu_12']
['Conv2D_13', 'dout_relu_12', 'output_data_13']
['relu_14', 'output_data_13', 'dout_relu_14']
['Conv2D_15', 'dout_relu_14', 'output_data_15']
['relu_16', 'output_data_15', 'dout_relu_16']
['max_pooling_3d_17', 'dout_relu_16', 'out_data_17']
['Conv2D_18', 'out_data_17', 'output_data_18']
['relu_19', 'output_data_18', 'dout_relu_19']
['Conv2D_20', 'dout_relu_19', 'output_data_20']
['relu_21', 'output_data_20', 'dout_relu_21']
['Conv2D_22', 'dout_relu_21